Setup

In [3]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

In [4]:
from context import Context

context = Context()

2023-12-04 14:13:52,168 - WARNING - Structure definition for stakings_boosted not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-testnet/deploy_structure.json
2023-12-04 14:13:52,171 - WARNING - Structure definition for metastakings_boosted not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-testnet/deploy_structure.json
2023-12-04 14:13:52,173 - WARNING - Structure definition for position_creator not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-testnet/deploy_structure.json


loaded 10 accounts from 1 PEM files.


2023-12-04 14:13:55,716 - DEBUG - Account.sync_nonce() done: 8
2023-12-04 14:13:56,783 - DEBUG - Account.sync_nonce() done: 8
2023-12-04 14:13:56,784 - INFO - Loaded 5 tokens.
2023-12-04 14:13:58,269 - DEBUG - Account.sync_nonce() done: 8
2023-12-04 14:13:58,270 - INFO - Loaded 1 egld_wraps.
2023-12-04 14:13:58,272 - INFO - Loaded 1 locked_assets.
2023-12-04 14:13:58,273 - INFO - Loaded 1 proxies.
2023-12-04 14:13:58,274 - INFO - Loaded 1 simple_locks.
2023-12-04 14:13:58,275 - INFO - Loaded 1 simple_locks_energy.
2023-12-04 14:13:58,276 - INFO - Loaded 1 fees_collectors.
2023-12-04 14:13:58,278 - INFO - Loaded 1 unstakers.
2023-12-04 14:13:58,279 - INFO - Loaded 1 proxies_v2.
2023-12-04 14:13:58,279 - INFO - Loaded 0 router.
2023-12-04 14:13:58,280 - INFO - Loaded 1 router_v2.
2023-12-04 14:13:58,281 - INFO - Loaded 0 pairs.
2023-12-04 14:13:58,283 - INFO - Loaded 4 pairs_v2.
2023-12-04 14:13:58,284 - INFO - Loaded 1 pairs_view.
2023-12-04 14:13:58,285 - INFO - Loaded 0 farms_communit

In [5]:
from contracts.egld_wrap_contract import EgldWrapContract
from utils.utils_chain import Account

token = 'WEGLD-dd8471'
contract = EgldWrapContract(token)
owner = Account(pem_file='~/Documents/sh2.pem')
owner.sync_nonce(context.network_provider.proxy)
hash, address = contract.contract_deploy(owner, context.network_provider.proxy, config.EGLD_WRAP_BYTECODE_PATH)

2023-12-04 14:14:00,211 - DEBUG - Account.sync_nonce() done: 3
2023-12-04 14:14:00,212 - INFO - deploy EgldWrapContract contract
2023-12-04 14:14:00,213 - DEBUG - Deploy EgldWrapContract
2023-12-04 14:14:00,922 - DEBUG - Deploy arguments: ['WEGLD-dd8471']
2023-12-04 14:14:01,851 - INFO - View this transaction in the MultiversX Testnet Explorer: https://testnet-explorer.multiversx.com/transactions/e0697ad40dcec159af117b0bac6e5196414cc4b8196625402f2938e2ca621d04


KeyboardInterrupt: 

In [ ]:
from contracts.builtin_contracts import ESDTContract

esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = esdt_contract.set_special_role_token(context.deployer_account, context.network_provider.proxy,
                                               [token, address, "ESDTRoleLocalMint", "ESDTRoleLocalBurn"])


In [ ]:
contract.address = address
contract.resume(owner, context.network_provider.proxy)

In [ ]:
from utils.utils_tx import endpoint_call
from utils.utils_chain import WrapperAddress

endpoint_call(context.network_provider.proxy, 50000000, owner, WrapperAddress(contract.address), "ChangeOwnerAddress", [context.deployer_account.address.bech32()])

Token transfer